In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tf.__version__

'2.15.0'

In [ ]:
!apt install ffmpeg
# !pip install -q spleeter
!pip install -q TeleBot
!pip install -q pyTelegramBotAPI
!pip install -q telegram
!pip install -q pydub
# !pip install tensorflow==2.15.0
# !pip install tensorflow-io-gcs-filesystem
# !pip install tensorflow-estimator

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install protobuf

In [ ]:
import os
import telebot
import telegram
import tensorflow as tf
import numpy as np
import pydub
from IPython.display import Audio
#from spleeter.separator import Separator


In [ ]:
bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
model = tf.keras.models.load_model("/content/drive/MyDrive/singers_folders/weights/friends_audio_classification_2.h5")
model2 = tf.keras.models.load_model("/content/drive/MyDrive/singers_folders/weights/ADAMAX_SIGNERS_weights.h5")



@bot.message_handler(commands=["start"])
def send_welcome(message):
    bot.send_message(message.chat.id , "Hello " + message.from_user.first_name + "\nWelcome to Audio Recognizer bot🎙🎵🔊" )
    help(message)

@bot.message_handler(commands=['help'])
def help(message):
    my_keyboard= telebot.types.ReplyKeyboardMarkup(row_width=3)
    key1 = telebot.types.KeyboardButton("/Friends_voice_recognition")
    key2 = telebot.types.KeyboardButton("/Singers_voice_recognition")
    my_keyboard.row(key1)
    my_keyboard.row(key2)
    bot.send_message(message.chat.id ,"Choose one of the below keys :" , reply_markup=my_keyboard)


@bot.message_handler(commands=['Friends_voice_recognition'])
def mymessage(message):
  message1 = bot.send_message(message.chat.id , "Now send a voice ... ")
  bot.register_next_step_handler(message1 , friends_voice_recognizer)

@bot.message_handler(content_types=['voice'])
def friends_voice_recognizer(message):
  file_id = message.voice.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/drive/MyDrive/singers_folders/data/voice.ogg", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return

  audio = pydub.AudioSegment.from_file("/content/drive/MyDrive/singers_folders/data/voice.ogg")
  audio_array = audio.get_array_of_samples()
  silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=2000 , silence_thresh=-45)
  result = sum(silent_chunks)
  result.export("/content/drive/MyDrive/singers_folders/wav_data/voice.wav")
  audio = pydub.AudioSegment.from_file("/content/drive/MyDrive/singers_folders/wav_data/voice.wav")
  chunks = pydub.utils.make_chunks(audio , 1000)
  for i , chunk in enumerate(chunks) :
      if len(chunk) >= 1000 :
          chunk.export(os.path.join("/content/drive/MyDrive/singers_folders/dataset/z/" , "fvoice.wav") , format="wav")


  test_data = tf.keras.utils.audio_dataset_from_directory(
    "/content/drive/MyDrive/singers_folders/dataset",
    shuffle=False,batch_size=8,output_sequence_length= 48000,ragged= False,
    labels="inferred",label_mode="categorical",sampling_rate= None,seed=59
  )

  classes_labels = np.array(['abdollah', 'azra', 'davood', 'javad', 'khadijeh', 'kiana','maryam', 'matin','melika', 'mohammad',
                             'mohammad_parvari','mona', 'nahid', 'nima', 'omid', 'parisa','parsa', 'sadegh', 'sajedeh', 'shima', 'tara'])
  predictions = model.predict(test_data)
  probability = predictions
  predicted_class_index = np.argmax(probability)
  print(predicted_class_index)
  predicted_class = classes_labels[predicted_class_index]
  print(predicted_class)
  print(f'The owner of this voice is : {predicted_class}')
  bot.send_message(chat_id= message.chat.id , text=f'The owner of this voice is  :  {predicted_class}')




#~~~~~~~~~~~~~~~~ singers audio recognition ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
@bot.message_handler(commands=['Singers_voice_recognition'])
def mymessage2(message):
  message2 = bot.send_message(message.chat.id , "Now send a song , and wait for 15 seconds ... ")
  bot.register_next_step_handler(message2 , singers_voice_recognition)

@bot.message_handler(content_types=['audio'])
def singers_voice_recognition(message):

  file_id = message.audio.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/drive/MyDrive/singers_folders/data/audio.mp3", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return


  for file in os.listdir("/content/audio"):
    file_name = os.path.basename(file)
    f = os.path.splitext(file_name)
    if f[0] == "vocals" :
      audio = pydub.AudioSegment.from_file(os.path.join("/content/audio" , file))
      silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=2000 , silence_thresh=-45)
      result = sum(silent_chunks)
      result.export("/content/drive/MyDrive/singers_folders/wav_data2/"+ file)

  for file in os.listdir("/content/audio"):
    file_name = os.path.basename(file)
    f = os.path.splitext(file_name)
    if f[0] == "vocals" :
      audio = pydub.AudioSegment.from_file(os.path.join("/content/audio" , file))
      chunks = pydub.utils.make_chunks(audio , 1000)
      # create a folder for each person
      person_name = file.split(".")[0]
      os.makedirs(os.path.join("/content/drive/MyDrive/singers_folders/dataset2" , person_name) , exist_ok=True)
      for i , chunk in enumerate(chunks) :
          if len(chunk) >= 1000 : # if len chunk is more than 1 second
              chunk.export(os.path.join("/content/drive/MyDrive/singers_folders/dataset2" , person_name , f"voice_{i}.wav") , format="wav")

  #choose a random 1 sec file :
  pred_list = []
  #for i , file in enumerate(os.listdir("/content/drive/MyDrive/singers_folders/dataset2/vocals")):
  file_path = "/content/audio/vocals.wav"#f"/content/drive/MyDrive/singers_folders/dataset2/vocals/voice_{i}.wav"
  x = tf.io.read_file(file_path)
  x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=48000,)
  x = tf.squeeze(x , axis=-1)
  x = x[tf.newaxis, ...]
  classes_labels = np.array(['chaartar', 'chavoshi', 'sirvan', 'xaniar', 'yegane'])
  predictions = model2.predict(x)
  predicted_class_index = np.argmax(predictions)
  predicted_class = classes_labels[predicted_class_index]
  pred_list.append(predicted_class)

  predicted_class = max(pred_list ,key=pred_list.count)
  print(f'The owner of this song is : {predicted_class} ')
  bot.send_message(chat_id= message.chat.id , text=f'The singer of this song is  :  {predicted_class}')



bot.infinity_polling()

1/1 [==============================] - 0s 152ms/step
singer: yegane
The owner of this song is : yegane 
1/1 [==============================] - 0s 23ms/step
The owner of this song is : yegane 
1/1 [==============================] - 0s 33ms/step
singer: sirvan
The owner of this song is : sirvan 
1/1 [==============================] - 0s 22ms/step
The owner of this song is : sirvan 
1/1 [==============================] - 0s 22ms/step
singer: xaniar
The owner of this song is : xaniar 
1/1 [==============================] - 0s 37ms/step
The owner of this song is : xaniar 
1/1 [==============================] - 0s 36ms/step
singer: chaartar
The owner of this song is : chaartar 
1/1 [==============================] - 0s 23ms/step
The owner of this song is : chaartar 
1/1 [==============================] - 0s 36ms/step
singer: sirvan
The owner of this song is : sirvan 
1/1 [==============================] - 0s 25ms/step
The owner of this song is : sirvan 


2023-12-30 08:34:05,270 (__init__.py:968 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-12-30 08:34:05,274 (__init__.py:970 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
